In [13]:
import cv2
import sys
import numpy
import os
#os.environ['QT_QPA_PLATFORM'] = 'xcb'

In [14]:
PREVIEW = 0 #Raw camera feed
BLUR = 1 #Blur the filter
FEATURES = 2 #corner detection
CANNY = 3 #apply canny edge detector filter

In image processing, "corners" refer to points where the intensity changes sharply in multiple directions. These are often found at:

1. The intersection of two edges (e.g., corners of objects in an image).
2. High-contrast texture points (e.g., checkerboard patterns, text edges).

In [15]:
feature_params = dict(maxCorners = 500, qualityLevel = 0.2, minDistance = 15, blockSize = 9)
#maxCorners = 500: Detect up to 500 corners.
#qualityLevel = 0.2: Minimum quality of corners (higher means fewer but stronger corners).
#minDistance = 15: Minimum distance between detected corners.
#blockSize = 9: Size of the neighborhood for corner detection.
s = 0

if len(sys.argv) > 1:
    s = sys.argv[1]

In [16]:
image_filter = PREVIEW
alive = True

win_name = 'Camera Filters'
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
result = None

source = cv2.VideoCapture(s)

[ WARN:0@60.130] global cap_gstreamer.cpp:1436 open OpenCV | GStreamer warning: Error opening bin: syntax error
[ WARN:0@60.130] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


### A video is essentially a sequence of images (frames) displayed rapidly, typically at 30 or 60 frames per second (FPS).

In your code, the loop continuously:

1. Captures a single image (frame) from the video source.
2. Processes it (flipping, filtering, etc.).
3. Displays it in real-time.
4. Repeats the process, creating the illusion of motion.

In [ ]:
#After videocapture, we read the capture frame by frame
while alive:
    has_frame, frame = source.read()
    #has_frame is True if a frame is successfully captured.
    #frame contains the image data
    if not has_frame:
        break
    frame = cv2.flip(frame, 1)

    if image_filter == PREVIEW: #Shows original image
        result = frame
    elif image_filter == CANNY: #Applies a canny edge detector filter. Detects edges by finding areas of strong intensity changes
        result = cv2.Canny(frame, 80, 150)
    elif image_filter == BLUR:
        result = cv2.blur(frame, (13, 13)) #Applies a blur effect. Smoothens image to reduce noise
    elif image_filter == FEATURES:
        result = frame
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #Converts the frame to grayscale (because corner detection works on intensity, not color).
        corners = cv2.goodFeaturesToTrack(frame_gray, **feature_params) #Detects corners using goodFeaturesToTrack function
        if corners is not None: #If corners are found, it loops through them and draws green circles around them.
            for x, y in numpy.float32(corners).reshape(-1, 2):
                cv2.circle(result, (int(x), int(y)), 10, (0, 255, 0), 1)

    cv2.imshow(win_name, result)

    key = cv2.waitKey(1) #Keeps window responsive, Without it, the OpenCV window may freeze.
    #waits for a key to be pressed for 1ms
    if key == ord("Q") or key == ord("q") or key == 27:
        alive = False
    elif key == ord("C") or key == ord("c"): #ord() gets the ASCII value of the key
        image_filter = CANNY
    elif key == ord("B") or key == ord("b"):
        image_filter = BLUR
    elif key == ord("F") or key == ord("f"):
        image_filter = FEATURES
    elif key == ord("P") or key == ord("p"):
        image_filter = PREVIEW


In [ ]:
source.release()
cv2.destroyWindow(win_name)

: 